In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [4]:
!pip install langchain_text_splitters

In [5]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.8 MB/s eta 0:00:00


In [6]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.8 MB/s eta 0:00:00


In [7]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [12]:
loader = PyPDFLoader("/content/GUIDE D’ASSURANCE DE LA PME.pdf")
documents = loader.load()
print("Le fichier a été uploadé")
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_documents(documents)
print("Le fichier divisé en plusieurs sous documents")

Le fichier a été uploadé
Le fichier divisé en plusieurs sous documents


In [13]:
# Generate embeddings and initialize a retriever
embeddings = OllamaEmbeddings(model="znbang/bge:large-en-v1.5-f16", show_progress=True)
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever()

OllamaEmbeddings: 100%|██████████| 34/34 [00:07<00:00,  4.34it/s]


In [14]:
# Create a prompt template
template = """Répondez à la question uniquement sur la base du contexte suivant:

    {context}

    Question: {question}
    """
prompt = ChatPromptTemplate.from_template(template)

In [15]:
# Define the LLM and the full chain
model = ChatOllama(model="phi3", num_predict=2000)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)


In [17]:
question= " De quoi parle ce livre ?  ? "
chain.invoke({"question": question})

OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.88s/it]


"Ce guide d'assurance de la PME au Maroc, avec le concours de la Conférence des Nations Unies sur le Commerce et le Développement (UNCTAD), parle de l’ensemble des assurances obligatoires pour une entreprise en Maroc, notamment l'Assurance Maladie Obligatoire, l'Assurance Accidents du Travail et l'assurance automobile. Il fournit également un aperçu plus large sur les types d'assurances qui sont utiles aux entreprises, à côté des assurances obligatoires, pour couvrir différents risques spécifiques liés à leur activité."

In [18]:
question= " quels sont les produits assurance pour PME ?  ? "
chain.invoke({"question": question})

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.04s/it]


"Les produits d'assurance obligatoires pour une entreprise en France incluent l'Assurance Maladie Obligatoire (AMO) au profit des salariés, l'Assurance Accidents du Travail et l'Assurance Responsabilité Civile liée à la possession de véhicules terrestres à moteur. Pour les produits d'assurance pour PME qui ne sont pas obligatoires mais considérés comme utiles, il est conseillé de consulter le guide d'assurance de la PME au Maroc et autres ressources disponibles pour évaluer les risques spécifiques à votre entreprise."